In [1]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import json
import pandas

## Spotipy setup

- Need client id and client secret which you can get from [here](https://developer.spotify.com/dashboard/login).
- Also need to setup your redirectURI. I just use Google. It doesn't really matter. You can do this by clicking on the green 'Edit Settings' button. Make sure to add and save.
- Username is your user id which you can find by (at least on the desktop app) clicking on your name in the top right to get to your profile, clicking the '...' under your name, hover over 'Share' and navigate to 'Copy Profile Link'. Click it. Paste somewhere to show the link which should be something like https://open.spotify.com/user/** where ** is your user id.

In [2]:
cId = "xxx"
cSecret = "xxx"
redirectURI = "https://google.com/"
username = "xxx"

client_credentials_manager = SpotifyClientCredentials(client_id=cId, client_secret=cSecret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

scope = 'user-library-read user-read-currently-playing user-read-playback-state user-read-recently-played'
token = util.prompt_for_user_token(username, scope, cId, cSecret, redirectURI)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

## Importing some saved tracks from spotify to a csv file

This stores the user's saved tracks into a list named track_list and the album info into album_list.

The "amount" is for the number of songs we want. I'm getting 100 for now. I have well over 1000 songs saved and it worked fine using that amount. I don't know how this would work if the user does not have more than the amount specified. 

In [4]:
track_list = []
album_list = []

limit = 50
amount = 100

for i in range(0, amount, limit):
    
    saved_tracks = sp.current_user_saved_tracks(limit=limit, offset=i)
    
    for t in saved_tracks['items']:
        track = t['track']
        
        track_id = track['id']
        track_name = track['name']
        
        track_artists = ''
        for a in track['artists']:
            track_artists += a['name'] + ','
        track_artists = track_artists[:-1]
        
        track_album = track['album']
        
        album_id = track_album['id']
        album_name = track_album['name']
        album_img = track_album['images'][0]['url']
        album_type = track_album['album_type']
        release_date = track_album['release_date']

        album_artists = ''
        for a in track_album['artists']:
            album_artists += a['name'] + ','
        album_artists = track_artists[:-1]
        
        track_dict = {
            'track_id': track_id,
            'track_name': track_name,
            'track_artists': track_artists,
            'album_id': album_id            
        }
        
        album_dict = {
            'album_id': album_id,
            'album_name': album_name,
            'album_img': album_img,
            'album_type': album_type,
            'release_date': release_date
        }
        
        track_list.append(track_dict)
        album_list.append(album_dict)

Converting the track list into a dataframe

In [5]:
track_df = pandas.DataFrame(track_list)
track_df.set_index('track_id', inplace=True) # change index to be the track id
track_df.head() # preview first 5 rows of data

,album_id,track_artists,track_name
track_id,,,
2RSOzvKUnfDDrb2nQPfpQU,7uYn7ZSrXOVlmITyocZs9F,Aqua,Barbie Girl
3DZQ6mzUkAdHqZWzqxBKIK,4eddbruVtOqw8khwxSH6H2,"Rihanna,Future",Loveeeeeee Song
68mxSfFGEVRJFoq2UCaVsU,6sGYOY400cCvztdQ4mBSZs,Catch 22,The Purge
2Y6MLh2iXf6PP2GzHPlLzb,102J7QdW6oFSUlQcWmJZ3E,Shoffy,Different Skies
6WmIyn2fx1PKQ0XDpYj4VR,6PWXKiakqhI17mTYM4y6oY,The 1975,Love It If We Made It


In [6]:
album_df = pandas.DataFrame(album_list)
album_df.set_index('album_id', inplace=True)
album_df.drop_duplicates(keep='first', inplace=True) # drop duplicate albums
album_df.head()

,album_img,album_name,album_type,release_date
album_id,,,,
7uYn7ZSrXOVlmITyocZs9F,https://i.scdn.co/image/842c8f1e41d2ceb29dfaa9...,Aquarium,album,1997-01-01
4eddbruVtOqw8khwxSH6H2,https://i.scdn.co/image/31ed5458991f5d4da0219c...,Unapologetic (Deluxe),album,2012-12-11
6sGYOY400cCvztdQ4mBSZs,https://i.scdn.co/image/dd1f727c82b7c4efcd0ab6...,Permanent Revolution,album,2006
102J7QdW6oFSUlQcWmJZ3E,https://i.scdn.co/image/be577ad43031a3615d349a...,Shoffy,album,2017-09-09
6PWXKiakqhI17mTYM4y6oY,https://i.scdn.co/image/24753fe5d9ec614b83e1f0...,A Brief Inquiry Into Online Relationships,album,2018-11-30


Checking dataframe shapes

In [7]:
# 100 rows of data
# 3 columns
print(track_df.shape)

# less than or equal to 100 rows of data
# 4 columns
print(album_df.shape)

(100, 3)
(66, 4)


Saving the dataframes as csv files

In [8]:
track_df.to_csv('songs.csv')
album_df.to_csv('albums.csv')

Check out the csv files!